# Lab 4: Vegetation and Evapotranspiration

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MichalBrezny/LSPD-lab/blob/main/labs/Lab04_Vegetation_Evapotranspiration.ipynb)

## Objectives
- Understand the components of evapotranspiration (ET)
- Calculate reference ET using the Penman-Monteith equation
- Analyze stomatal conductance and its controls
- Explore the relationship between vegetation and water/energy fluxes

## Background
Evapotranspiration is the combined process of evaporation from soil and water surfaces and transpiration from plants. It is a key component of the water and energy cycles, linking the land surface with the atmosphere.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Physical constants
LATENT_HEAT = 2.45e6  # J/kg (at 20°C)
SPECIFIC_HEAT = 1005  # J/(kg·K)
AIR_DENSITY = 1.2  # kg/m³
PSYCHROMETRIC = 0.067  # kPa/°C

print("Libraries imported successfully!")

## Part 1: Penman-Monteith Equation

The Penman-Monteith equation calculates evapotranspiration from energy balance and aerodynamic considerations:

$LE = \frac{\Delta(R_n - G) + \rho_a c_p \frac{VPD}{r_a}}{\Delta + \gamma(1 + r_s/r_a)}$

where:
- $\Delta$ = slope of saturation vapor pressure curve
- $R_n$ = net radiation
- $G$ = ground heat flux
- $\rho_a$ = air density
- $c_p$ = specific heat of air
- $VPD$ = vapor pressure deficit
- $r_a$ = aerodynamic resistance
- $r_s$ = surface (stomatal) resistance
- $\gamma$ = psychrometric constant

In [ ]:
def saturation_vapor_pressure(T):
    """
    Calculate saturation vapor pressure using Tetens equation.
    
    Parameters:
    - T: temperature in °C
    
    Returns:
    - es: saturation vapor pressure in kPa
    """
    return 0.611 * np.exp(17.27 * T / (T + 237.3))

def vapor_pressure_deficit(T, RH):
    """
    Calculate vapor pressure deficit.
    
    Parameters:
    - T: temperature in °C
    - RH: relative humidity (0-1)
    
    Returns:
    - VPD: vapor pressure deficit in kPa
    """
    es = saturation_vapor_pressure(T)
    ea = RH * es
    return es - ea

def slope_svp_curve(T):
    """
    Calculate slope of saturation vapor pressure curve.
    
    Parameters:
    - T: temperature in °C
    
    Returns:
    - Delta: slope in kPa/°C
    """
    es = saturation_vapor_pressure(T)
    return 4098 * es / (T + 237.3)**2

def aerodynamic_resistance(u, z=2, z0=0.1, d=0):
    """
    Calculate aerodynamic resistance.
    
    Parameters:
    - u: wind speed at height z (m/s)
    - z: measurement height (m)
    - z0: roughness length (m)
    - d: displacement height (m)
    
    Returns:
    - ra: aerodynamic resistance (s/m)
    """
    k = 0.41  # von Karman constant
    if u < 0.1:
        u = 0.1  # Avoid very low wind speeds
    return np.log((z - d) / z0)**2 / (k**2 * u)

def penman_monteith(Rn, G, T, RH, u, rs=70):
    """
    Calculate evapotranspiration using Penman-Monteith equation.
    
    Parameters:
    - Rn: net radiation (W/m²)
    - G: ground heat flux (W/m²)
    - T: air temperature (°C)
    - RH: relative humidity (0-1)
    - u: wind speed (m/s)
    - rs: surface resistance (s/m)
    
    Returns:
    - LE: latent heat flux (W/m²)
    - ET: evapotranspiration rate (mm/hr)
    """
    # Calculate components
    Delta = slope_svp_curve(T)
    VPD = vapor_pressure_deficit(T, RH)
    ra = aerodynamic_resistance(u)
    gamma = PSYCHROMETRIC
    
    # Penman-Monteith equation
    numerator = Delta * (Rn - G) + AIR_DENSITY * SPECIFIC_HEAT * VPD / ra
    denominator = Delta + gamma * (1 + rs / ra)
    
    LE = numerator / denominator
    
    # Convert to mm/hr
    ET = LE * 3600 / LATENT_HEAT  # W/m² to mm/hr
    
    return LE, ET

print("Penman-Monteith functions defined!")

In [ ]:
# Generate diurnal cycle data
hours = np.arange(0, 24, 0.5)

# Meteorological conditions
Rn = 400 * np.sin(np.pi * (hours - 6) / 12)
Rn[Rn < 0] = -50
G = 0.1 * Rn

T = 20 + 8 * np.sin(np.pi * (hours - 8) / 12)
RH = 0.6 - 0.2 * np.sin(np.pi * (hours - 8) / 12)
RH = np.clip(RH, 0.3, 0.9)

u = 2 + 1 * np.sin(np.pi * (hours - 12) / 12)
u = np.abs(u)

# Different vegetation types
vegetation_types = {
    'Grass': 70,
    'Crops': 50,
    'Forest': 150,
    'Desert': 500
}

et_results = {}
for veg_name, rs in vegetation_types.items():
    LE = np.zeros_like(hours)
    ET = np.zeros_like(hours)
    
    for i in range(len(hours)):
        LE[i], ET[i] = penman_monteith(Rn[i], G[i], T[i], RH[i], u[i], rs)
    
    et_results[veg_name] = {'LE': LE, 'ET': ET}

print("ET calculations complete!")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Meteorological inputs
ax = axes[0, 0]
ax2 = ax.twinx()
ax.plot(hours, T, 'r-', linewidth=2, label='Temperature')
ax2.plot(hours, RH * 100, 'b--', linewidth=2, label='Relative Humidity')
ax.set_xlabel('Hour of Day', fontsize=11)
ax.set_ylabel('Temperature (°C)', fontsize=11, color='r')
ax2.set_ylabel('Relative Humidity (%)', fontsize=11, color='b')
ax.set_title('Meteorological Conditions', fontsize=12, fontweight='bold')
ax.tick_params(axis='y', labelcolor='r')
ax2.tick_params(axis='y', labelcolor='b')
ax.grid(True, alpha=0.3)

# VPD
ax = axes[0, 1]
VPD = vapor_pressure_deficit(T, RH)
ax.plot(hours, VPD, 'purple', linewidth=2)
ax.fill_between(hours, 0, VPD, alpha=0.3, color='purple')
ax.set_xlabel('Hour of Day', fontsize=11)
ax.set_ylabel('VPD (kPa)', fontsize=11)
ax.set_title('Vapor Pressure Deficit', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# Latent heat flux
ax = axes[1, 0]
for veg_name, data in et_results.items():
    ax.plot(hours, data['LE'], linewidth=2, label=veg_name)
ax.set_xlabel('Hour of Day', fontsize=11)
ax.set_ylabel('Latent Heat Flux (W/m²)', fontsize=11)
ax.set_title('Latent Heat Flux by Vegetation Type', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# ET rate
ax = axes[1, 1]
for veg_name, data in et_results.items():
    ax.plot(hours, data['ET'], linewidth=2, label=veg_name)
ax.set_xlabel('Hour of Day', fontsize=11)
ax.set_ylabel('ET Rate (mm/hr)', fontsize=11)
ax.set_title('Evapotranspiration Rate by Vegetation Type', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Part 2: Stomatal Conductance

Stomatal conductance controls plant transpiration and varies with environmental conditions:

$g_s = g_{max} \cdot f(PAR) \cdot f(VPD) \cdot f(T) \cdot f(\theta)$

where functions represent limitations by:
- PAR: Photosynthetically Active Radiation
- VPD: Vapor Pressure Deficit
- T: Temperature
- θ: Soil moisture

In [ ]:
def stomatal_conductance(PAR, VPD, T, theta, 
                         gs_max=0.01, 
                         PAR_half=400, 
                         VPD_max=3, 
                         T_opt=25,
                         theta_min=0.15):
    """
    Calculate stomatal conductance based on environmental factors.
    
    Parameters:
    - PAR: photosynthetically active radiation (μmol/m²/s)
    - VPD: vapor pressure deficit (kPa)
    - T: temperature (°C)
    - theta: soil moisture (m³/m³)
    - gs_max: maximum stomatal conductance (m/s)
    
    Returns:
    - gs: stomatal conductance (m/s)
    """
    # PAR limitation (Michaelis-Menten)
    f_PAR = PAR / (PAR + PAR_half)
    
    # VPD limitation (linear decrease)
    f_VPD = np.maximum(0, 1 - VPD / VPD_max)
    
    # Temperature limitation (parabolic)
    f_T = 1 - ((T - T_opt) / 20)**2
    f_T = np.clip(f_T, 0, 1)
    
    # Soil moisture limitation
    f_theta = np.maximum(0, (theta - theta_min) / (0.35 - theta_min))
    f_theta = np.clip(f_theta, 0, 1)
    
    gs = gs_max * f_PAR * f_VPD * f_T * f_theta
    
    return gs, f_PAR, f_VPD, f_T, f_theta

# Simulate stomatal conductance
PAR = 1000 * np.maximum(0, np.sin(np.pi * (hours - 6) / 12))
theta_soil = 0.25  # constant for now

gs = np.zeros_like(hours)
factors = {'PAR': [], 'VPD': [], 'T': [], 'theta': []}

for i in range(len(hours)):
    gs[i], f_PAR, f_VPD, f_T, f_theta = stomatal_conductance(
        PAR[i], VPD[i], T[i], theta_soil
    )
    factors['PAR'].append(f_PAR)
    factors['VPD'].append(f_VPD)
    factors['T'].append(f_T)
    factors['theta'].append(f_theta)

for key in factors:
    factors[key] = np.array(factors[key])

print("Stomatal conductance calculated!")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# PAR
ax = axes[0, 0]
ax.plot(hours, PAR, 'gold', linewidth=2)
ax.fill_between(hours, 0, PAR, alpha=0.3, color='gold')
ax.set_xlabel('Hour of Day', fontsize=11)
ax.set_ylabel('PAR (μmol/m²/s)', fontsize=11)
ax.set_title('Photosynthetically Active Radiation', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# Stomatal conductance
ax = axes[0, 1]
ax.plot(hours, gs * 1000, 'green', linewidth=2)  # Convert to mm/s
ax.fill_between(hours, 0, gs * 1000, alpha=0.3, color='green')
ax.set_xlabel('Hour of Day', fontsize=11)
ax.set_ylabel('Stomatal Conductance (mm/s)', fontsize=11)
ax.set_title('Stomatal Conductance', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# Limitation factors
ax = axes[1, 0]
ax.plot(hours, factors['PAR'], linewidth=2, label='Light')
ax.plot(hours, factors['VPD'], linewidth=2, label='VPD')
ax.plot(hours, factors['T'], linewidth=2, label='Temperature')
ax.plot(hours, factors['theta'], linewidth=2, label='Soil Moisture')
ax.set_xlabel('Hour of Day', fontsize=11)
ax.set_ylabel('Limitation Factor (0-1)', fontsize=11)
ax.set_title('Environmental Limitation Factors', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Combined limitation
ax = axes[1, 1]
combined = factors['PAR'] * factors['VPD'] * factors['T'] * factors['theta']
ax.plot(hours, combined, 'darkgreen', linewidth=2)
ax.fill_between(hours, 0, combined, alpha=0.3, color='darkgreen')
ax.set_xlabel('Hour of Day', fontsize=11)
ax.set_ylabel('Combined Limitation (0-1)', fontsize=11)
ax.set_title('Combined Environmental Limitation', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Exercises

### Exercise 1: Daily ET
Calculate the total daily evapotranspiration (mm/day) for each vegetation type.

In [ ]:
# Your code here
print("Daily Evapotranspiration by Vegetation Type:")
print("="*50)

for veg_name, data in et_results.items():
    daily_ET = np.sum(data['ET']) * 0.5  # 0.5 hour timestep
    print(f"{veg_name:15s}: {daily_ET:.1f} mm/day")

print("\nNote: Desert vegetation has lowest ET due to high surface resistance.")

### Exercise 2: Water Use Efficiency
Calculate the ratio of daily ET to net radiation (ET/Rn) for grass vegetation.

In [ ]:
# Your code here
# Convert ET to energy units (W/m²)
LE_grass = et_results['Grass']['LE']
Rn_positive = Rn.copy()
Rn_positive[Rn_positive < 0] = 0

# Daily totals
daily_LE = np.sum(LE_grass[Rn > 0]) * 0.5 * 3600  # Wh/m²
daily_Rn = np.sum(Rn_positive) * 0.5 * 3600  # Wh/m²

ratio = daily_LE / daily_Rn

print(f"Daily LE (grass): {daily_LE:.0f} Wh/m²")
print(f"Daily Rn: {daily_Rn:.0f} Wh/m²")
print(f"LE/Rn ratio: {ratio:.2f}")
print(f"\nThis means {ratio*100:.0f}% of net radiation is used for evapotranspiration.")

## Discussion Questions

1. **Why does forest have higher ET than desert despite similar radiation?**
   - Lower surface resistance (stomata more open)
   - Greater access to deep soil water
   - Higher leaf area

2. **How does VPD affect transpiration?**
   - Higher VPD increases evaporative demand
   - But also triggers stomatal closure
   - Net effect depends on plant water status

3. **What is the difference between potential and actual ET?**
   - Potential ET: with unlimited water supply
   - Actual ET: limited by water availability
   - Difference indicates water stress

4. **Why is the Penman-Monteith equation important?**
   - Combines energy balance and aerodynamic approaches
   - Accounts for vegetation characteristics
   - Widely used in weather, climate, and hydrological models

## Summary

In this lab, you have:
- Calculated ET using the Penman-Monteith equation
- Analyzed the effect of vegetation type on ET
- Modeled stomatal conductance responses to environment
- Explored the controls on transpiration

## Next Steps
In Lab 5, we will integrate all components to analyze surface fluxes and their interactions with the atmosphere.